## Gemini pro for identify road updates in urls batches using Prompt Engineering.

In [27]:
!pip install -q -U google-generativeai
#!pip install datasets #transformers

In [28]:
#import pathlib
#import textwrap
import numpy as np
from bs4 import BeautifulSoup

import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from google.colab import userdata
from tqdm.notebook import tqdm

import pandas as pd
import requests
import time

### Url dataset

In [3]:
df = pd.read_excel('/content/drive/MyDrive/fuis/Articulos ArgChil.xlsx',header = None, sheet_name = 1,usecols = [2])

### Gemeni model and safety settings

In [29]:
model = genai.GenerativeModel('gemini-pro')
safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    }
GOOGLE_API_KEY=userdata.get('APIKEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [12]:
def url_extractL(df,doc):
  url = df.iloc[doc].to_numpy()[0]
  try:
    response = requests.get(url)
    if response.status_code == 404:
      return(404)
    if response.status_code == 200:
      return(BeautifulSoup(response.text, "lxml").text)
    else:
      print("URL bloqueada",doc)
      return(-2)
  except:
    print("URL no válida, url numero: ",doc)
    return(-1)

### *url_extract()* for extract usefull text for url index in *df* dataset

In [30]:
url_extractL(df,5)

'Caminos Río Uruguay propone reparar la Ruta 14 atado a una adecuación de tarifas – Diario El Argentino de Gualeguaychú. Edición onlineAdsCiudadInfo. GeneralDepartamentoFúnebresSocialesPolicialesDeportesProvincialesNacionalesInternacionalesEdición ImpresaSeccionesSuscribiteSuscribiteIngresáSeccionesviernes 03 de mayo de 2024SuscribiteIngresáSuscribiteIngresáVolverA+A-SuscribiteSuscribiteÚltimas noticiasEdición ImpresaCiudadFúnebresInfo. GeneralDepartamentoSocialesPolicialesDeportesProvincialesNacionalesInternacionalesLínea DirectaTiempo Facebook Twitter SuscribiteIngresarFacebookTwitterAdsProvinciales19 de abril de 2024 | 14.34 (actualizado hace 14 días)LA EMPRESA CONFIRMÓ LA PRÓRROGA DE LA CONCESIÓN POR UN AÑOCaminos Río Uruguay propone reparar la Ruta 14 atado a una adecuación de tarifasAdsTras las constantes críticas por el estado de la Autovía 14 y luego de las últimas gestiones a nivel legislativo, el vocero de la empresa Caminos del Río Uruguay, Mariano Bradanini, brindó explicac

### Prompt engineering

In [32]:
print(df.iloc[21].to_numpy())
text = url_extractL(df,21)
prompt = "Analiza siguiente texto buscando potenciales actualizaciones viales:\n"+ "'" + str(text) + "'\n" + """
      Resume el texto en un par de enunciados cortos y concretos y si el texto realmente contiene potenciales cambios viales,
      enumera los cambios viables que el texto especifica.
      """
response = model.generate_content(prompt,safety_settings=safety_settings)
print(response.text)

['https://www.diariojornada.com.ar/367254/politica/torres_y_bowen_inauguraron_luminarias_en_el_acceso_de_dolavon']
**Resumen:**

El texto anuncia la inauguración de nuevas luminarias en el acceso a Dolavon.

**Cambios viales potenciales:**

El texto no especifica cambios viales potenciales.


In [ ]:
resumen = []
for i in tqdm(range(len(df))):
  text = url_extractL(df,i)
  if text == 404: resumen.append("Pagina no encontrada")
  elif text == -1: resumen.append("Error de URL")
  elif text == -2: resumen.append("Bloqueo de URL")

  else:
    prompt = "Analiza siguiente texto buscando potenciales actualizaciones viales:\n"+ "'" + text + "'\n" + """
      Resume el texto en un par de enunciados cortos y concretos y si el texto realmente contiene potenciales cambios viales,
      enumera los cambios viables que el texto especifica.
      """
    response = model.generate_content(prompt,safety_settings=safety_settings)
    time.sleep(1)
    if str(response.candidates[0].finish_reason) == "FinishReason.OTHER":
      resumen.append("Error de Modelo de lenguaje")
    else:
      resumen.append(response.text)

In [ ]:
chi = np.append(progreso,resumen)
print(chi.shape,len(chi),"\n\n",chi[-1])
progreso = chi
np.save("/content/drive/MyDrive/fuis/progreso",progreso)

(1010,) 1010 

 **Resumen:**

* Las autoridades inauguraron 13 viviendas y entregaron apoyo a bibliotecas y clubes deportivos en Arrufó.
* Se anunciaron importantes proyectos viales, incluida la construcción de una rotonda y la repavimentación de varias carreteras.

**Cambios viales potenciales:**

* Construcción de una rotonda en la intersección de RN34 y RP39
* Repavimentación de tramos de RP39, RP23 y RP13
* Pavimentación del acceso al barrio Palermo en San Cristóbal
* Ensanchamiento de 3 puentes sobre la RP4


In [ ]:
progreso = np.load("/content/drive/MyDrive/fuis/progreso.npy")

In [ ]:
df.insert(1,"3",value = progreso)
df.to_excel("/content/drive/MyDrive/fuis/output.xlsx")

In [ ]:
# If the response doesn't contain text, check if the prompt was blocked.
print(response.prompt_feedback)
# Also check the finish reason to see if the response was blocked.
print(response.candidates[0].finish_reason)
# If the finish reason was SAFETY, the safety ratings have more details.
print(response.candidates[0].safety_ratings)


FinishReason.OTHER
[]
